# HITS

In this notebook, we will use both NetworkX and cuGraph to compute HITS.  
The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Created:   06/09/2020
* Updated:   06/22/2022

RAPIDS Versions: 0.15  

Test Hardware

* Tesla V100 32G, CUDA 11.5


## Introduction
HITS, also known as hubs and authorities, computes the relative importance of vertices.   

See [Wikipedia](https://en.wikipedia.org/wiki/HITS_algorithm) for more details on the algorithm.

HITS can be thought of as similar to PageRank, it is an iterative algorithm that propagates scores until a tolerance is reached, or max number of iterations is processed.  

---
From Wikepedia:

The algorithm performs a series of iterations, each consisting of two basic steps:

__Authority__ update: Update each node's authority score to be equal to the sum of the hub scores of each node that points to it. That is, a node is given a high authority score by being linked from pages that are recognized as Hubs for information.

__Hub__ update: Update each node's hub score to be equal to the sum of the authority scores of each node that it points to. That is, a node is given a high hub score by linking to nodes that are considered to be authorities on the subject.

---


To compute the HITS scores for a graph in cuGraph we use:<br>

**cugraph.hits(G,max_iter=100,
         tol=1.0e-5,
         nstart=None,
         normalized=True)**
* __G__: cugraph.Graph object
* __max_iter__: int, The maximum number of iterations before an answer is returned. 
* __tol__: float, Set the tolerance the approximation, this parameter should be a small magnitude value. 
* __nstart__: cudf.DataFrame - Not currently supported
* __normalized_ : bool - Not currently supported, always used as True

Returns:
* __df__: a cudf.DataFrame object with two columns:
  * df['vertex'] : cudf.Series
        Contains the vertex identifiers
  * df['hubs'] : cudf.Series
        Contains the hubs score
  * df['authorities'] : cudf.Series
        Contains the authorities score


### Some notes about vertex IDs...
* The current version of cuGraph requires that vertex IDs be representable as 32-bit integers, meaning graphs currently can contain at most 2^32 unique vertex IDs. However, this limitation is being actively addressed and a version of cuGraph that accommodates more than 2^32 vertices will be available in the near future.
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [ ]:
# The notebook compares cuGraph to NetworkX,  
# therefore there some additional non-RAPIDS python libraries need to be installed. 
# Please run this cell if you need the additional libraries
!pip install networkx

In [1]:
# Import needed libraries
import cugraph
import cudf

In [2]:
# NetworkX libraries
import networkx as nx

### Some Prep

In [3]:
# Import a built-in dataset
from cugraph.experimental.datasets import karate

---
# NetworkX

In [4]:
# Read the data, this also created a NetworkX Graph 
datafile = "../data/karate-data.csv"
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [5]:
hits_nx = nx.hits(Gnx)

In [6]:
hits_nx

({'1': 0.07141272880825199,
  '2': 0.05342723123553,
  '3': 0.06371906455637479,
  '4': 0.042422737124709,
  '5': 0.015260959706207484,
  '6': 0.015966913503059645,
  '7': 0.01596691350305965,
  '8': 0.03434316721905365,
  '9': 0.0456819251197503,
  '11': 0.015260959706207488,
  '12': 0.010617891511071221,
  '13': 0.016925450792306864,
  '14': 0.045494864068056355,
  '18': 0.018561637037432084,
  '20': 0.02971333388643479,
  '22': 0.018561637037432084,
  '32': 0.03837574186295603,
  '31': 0.035106237976714375,
  '10': 0.020625667749388635,
  '28': 0.026813494117104764,
  '29': 0.026331505777953887,
  '33': 0.06200184647383096,
  '17': 0.004748031847301575,
  '34': 0.07500294215657549,
  '15': 0.020370345825614266,
  '16': 0.020370345825614266,
  '19': 0.020370345825614266,
  '21': 0.020370345825614266,
  '23': 0.020370345825614266,
  '24': 0.03015649750935638,
  '26': 0.011893664396281384,
  '30': 0.027111539628217662,
  '25': 0.011460952230971702,
  '27': 0.015182734330338381},
 {'1':

Running NetworkX is that easy.  
Let's see how that compares to cuGraph

----

# cuGraph

### Create a Graph 

In [7]:
G = karate.get_graph()

### Call the HITS algorithm

In [8]:
# Call cugraph.hits to get the hits scores
gdf_hits = cugraph.hits(G)

_It was that easy!_  
Compared to NetworkX, the cuGraph data loading might have been more steps, but using cuDF allows for a wider range of data to be loaded. 


----

Let's now look at the results

In [9]:
def print_hub_threshold(_df, t=0) :
    filtered = _df.query('hubs >= @t')
    
    for i in range(len(filtered)):
        print("Best vertex is " + str(filtered['vertex'].iloc[i]) + 
            " with HUB score of " + str(filtered['hubs'].iloc[i]))  

In [10]:
def print_authorities_threshold(_df, t=0) :
    filtered = _df.query('authorities >= @t')
    
    for i in range(len(filtered)):
        print("Best vertex is " + str(filtered['vertex'].iloc[i]) + 
            " with Authorities score of " + str(filtered['authorities'].iloc[i]))  

In [11]:
print_hub_threshold(gdf_hits, gdf_hits['hubs'].max())

Best vertex is 34 with HUB score of 0.07500292228699207


In [12]:
print_authorities_threshold(gdf_hits, gdf_hits['authorities'].max())

Best vertex is 34 with Authorities score of 0.07500298081637535


----

Since this is a very small graph, let's just sort and get the first three records

In [13]:
sort_a = gdf_hits.sort_values('authorities', ascending=False)

In [14]:
sort_a.head(3)

,hubs,authorities,vertex
16,0.075003,0.075003,34
17,0.071413,0.071413,1
19,0.063719,0.063719,3


<img src="../img/zachary_graph_hits.png" width="600">

In [15]:
sort_h = gdf_hits.sort_values('hubs', ascending=False)

In [16]:
sort_h.head(32)

,hubs,authorities,vertex
16,0.075003,0.075003,34
17,0.071413,0.071413,1
19,0.063719,0.063719,3
18,0.062002,0.062002,33
20,0.053427,0.053427,2
23,0.045682,0.045682,9
24,0.045495,0.045495,14
21,0.042423,0.042423,4
22,0.038376,0.038376,32
30,0.035106,0.035106,31


___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___